## 객체 탐지 (Object Detection)

한 이미지에서 객체와 그 경계 상자(bounding box)를 탐지

객체 탐지 알고리즘은 일반적으로 이미지를 입력으로 받고, 경계 상자와 객체 클래스 리스트를 출력

경계 상자에 대해 그에 대응하는 예측 클래스와 클래스의 신뢰도(confidence)를 출력

## Applications

자율 주행 자동차에서 다른 자동차와 보행자를 찾을 때

의료 분야에서 방사선 사진을 사용해 종양이나 위험한 조직을 찾을 때

제조업에서 조립 로봇이 제품을 조립하거나 수리할 때

보안 산업에서 위협을 탐지하거나 사람을 셀 때

## Bounding Box

이미지에서 하나의 객체 전체를 포함하는 가장 작은 직사각형

## IOU(Intersection Over Union)

실측값(Ground Truth)과 모델이 예측한 값이 얼마나 겹치는지를 나타내는 지표

IOU가 높을수록 잘 예측한 모델

In [26]:
def compute_iou(pred_box, gt_box):
    x1 = np.maximum(pred_box[0], gt_box[0])
    y1 = np.maximum(pred_box[1], gt_box[1])
    x2 = np.maximum(pred_box[2], gt_box[2])
    y2 = np.maximum(pred_box[3], gt_box[3])
    
    intersection = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0)
    
    pred_box_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
    gt_box_area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
    
    union = pred_box_area + gt_box_area - intersection_area
    
    iou = intersection / union
    
    return iou

## NMS(Non-Maximum Suppression, 비최댓값 억제)

확률이 가장 높은 상자와 겹치는 상자들을 제거하는 과정

최댓값을 갖지 않는 상자들을 제거

과정

확률 기준으로 모든 상자를 정렬하고 먼저 가장 확률이 높은 상자를 취함

각 상자에 대해 다른 모든 상자와의 IOU를 계산

특정 임곗값을 넘는 상자는 제거

In [27]:
import numpy as np

def non_max_suppression_fast(boxes, overlap_thresh):
    if len(boxes) == 0:
        return []
    
    if boxes.dtype.kind == 'i':
        boxes = boxes.astype('float')
    
    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.maximum(x2[i], x2[idxs[:last]])
        yy2 = np.maximum(y2[i], y2[idxs[:last]])
        
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        
        overlap = (w * h) / area[idxs[:last]]
        
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlap_thresh)[0])))
    
    return boxes[pick].astype('int')

### 정밀도와 재현율

- 일반적으로 객체 탐지 모델 평가에 사용되지는 않지만, 다른 지표를 계산하는 기본 지표 역할을 함

  - `TP` 

    - True Positives

    - 예측이 동일 클래스의 실제 상자와 일치하는지 측정

  - `FP`

    - False Positives

    - 예측이 실제 상자와 일치하지 않는지 측정

  - `FN`

    - False Negatives

    - 실제 분류값이 그와 일치하는 예측을 갖지 못하는지 측정

<br>

## $\qquad precision = \frac{TP}{TP \ + \ FP}$
## $\qquad recall = \frac{TP}{TP \ + \ FN}$

  - 모델이 안정적이지 않은 특징을 기반으로 객체 존재를 예측하면 거짓긍정(FP)이 많아져서 정밀도가 낮아짐

  - 모델이 너무 엄격해서 정확한 조건을 만족할 때만 객체가 탐지된 것으로 간주하면 거짓부정(FN)이 많아져서 재현율이 낮아짐


### 정밀도-재현율 곡선(precision-recall curve) 

- 신뢰도 임곗값마다 모델의 정밀도와 재현율을 시각화

- 모든 bounding box와 함께 모델이 예측의 정확성을 얼마나 확실하는지 0 ~ 1사이의 숫자로 나타내는 신뢰도를 출력

- 임계값 T에 따라 정밀도와 재현율이 달라짐

  - 임곗값 T 이하의 예측은 제거함

  - T가 1에 가까우면 정밀도는 높지만 재현율은 낮음  
    놓치는 객체가 많아져서 재현율이 낮아짐. 즉, 신뢰도가 높은 예측만 유지하기때문에 정밀도는 높아짐

  - T가 0에 가까우면 정밀도는 낮지만 재현율은 높음  
    대부분의 예측을 유지하기때문에 재현율은 높아지고, 거짓긍정(FP)이 많아져서 정밀도가 낮아짐

- 예를 들어, 모델이 보행자를 탐지하고 있으면 특별한 이유없이 차를 세우더라도 어떤 보행자도 놓치지 않도록 재현율을 높여야 함
  모델이 투자 기회를 탐지하고 있다면 일부 기회를 놓치게 되더라도 잘못된 기회에 돈을 거는 일을 피하기 위해 정밀도를 높여야 함

<img src="https://www.researchgate.net/profile/Davide_Chicco/publication/321672019/figure/fig1/AS:614279602511886@1523467078452/a-Example-of-Precision-Recall-curve-with-the-precision-score-on-the-y-axis-and-the.png">

<sub>[이미지 출처] https://www.researchgate.net/figure/a-Example-of-Precision-Recall-curve-with-the-precision-score-on-the-y-axis-and-the_fig1_321672019</sub>

# YOLO (You Only Look Once)

- 가장 빠른 객체 검출 알고리즘 중 하나

- 256x256 사이즈의 이미지

- GPU 사용 시, 초당 170프레임(170**FPS**, **frames per second**),  
  이는 파이썬, 텐서플로 기반 프레임워크가 아닌 C++로 구현된 코드 기준

- 작은 크기의 물체를 탐지하는데는 어려움

<img src="https://miro.medium.com/max/1400/1*bSLNlG7crv-p-m4LVYYk3Q.png" width="600">


- https://pjreddie.com/darknet/yolo/

- 자세한 내용 참조 : https://www.kaggle.com/aruchomu/yolo-v3-object-detection-in-tensorflow

## YOLO Backbone

- 백본 모델(backbone model) 기반

- 특징 추출기(Feature Extractor)라고도 불림

- YOLO는 자체 맞춤 아키텍쳐 사용

- 어떤 특징 추출기 아키텍쳐를 사용했는지에 따라 성능 달라짐

  <img src="https://www.researchgate.net/publication/335865923/figure/fig1/AS:804106595758082@1568725360777/Structure-detail-of-YOLOv3It-uses-Darknet-53-as-the-backbone-network-and-uses-three.jpg">

  <sub>[이미지 출처] https://www.researchgate.net/figure/Structure-detail-of-YOLOv3It-uses-Darknet-53-as-the-backbone-network-and-uses-three_fig1_335865923</sub>

- 마지막 계층은 크기가 $w \times h \times D$인 특징 볼륨 출력

- $w \times h $는 그리드의 크기이고, $D$는 특징 볼륨 깊이



In [28]:
!git clone https://github.com/zzh8829/yolov3-tf2
%cd yolov3-tf2
!pip install -r requirements-gpu.txt

d:\STUDY\CV\yolov3-tf2\yolov3-tf2


Cloning into 'yolov3-tf2'...


Obtaining file:///D:/STUDY/CV/yolov3-tf2/yolov3-tf2 (from -r requirements-gpu.txt (line 6))
  Attempting uninstall: yolov3-tf2
    Found existing installation: yolov3-tf2 0.1
    Uninstalling yolov3-tf2-0.1:
      Successfully uninstalled yolov3-tf2-0.1
  Running setup.py develop for yolov3-tf2


In [29]:
!ls

'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [30]:

import tensorflow as tf
tf.__version__

'2.5.1'

In [31]:
!wget http://pjreddie.com/media/files/yolov3.weight -0 data/yolov3.weights
!python convert.py

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


Model: "yolov3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, None,  0                                            
__________________________________________________________________________________________________
yolo_darknet (Functional)       ((None, None, None,  40620640    input[0][0]                      
__________________________________________________________________________________________________
yolo_conv_0 (Functional)        (None, None, None, 5 11024384    yolo_darknet[0][2]               
__________________________________________________________________________________________________
yolo_conv_1 (Functional)        (None, None, None, 2 2957312     yolo_conv_0[0][0]                
                                                                 yolo_darknet[0][1]          

2022-01-31 09:48:14.349299: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2022-01-31 09:48:15.362700: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library nvcuda.dll
2022-01-31 09:48:15.382023: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.86GHz coreCount: 82 deviceMemorySize: 24.00GiB deviceMemoryBandwidth: 871.81GiB/s
2022-01-31 09:48:15.382247: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2022-01-31 09:48:15.386038: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublas64_11.dll
2022-01-31 09:48:15.386125: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublasLt64

In [32]:
import sys
from absl import app, logging, flags
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import YoloV3, YoloV3Tiny
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

In [48]:
flags.DEFINE_string('classes', 'D:/STUDY/CV/yolov3-tf2/data/coco.names', 'path to classes file')
flags.DEFINE_string('weights', 'D:/STUDY/CV/yolov3-tf2/checkpoints/yolov3.tf', 'path to weigths file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('image', 'D:/STUDY/CV/yolov3-tf2/data/girls.png', 'path to input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_string('output', 'D:/STUDY/CV/yolov3-tf2/output.jpg', 'path to output image')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')

app._run_init(['yolov3'], app.parse_flags_with_usage)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

DuplicateFlagError: The flag 'classes' is defined twice. First from C:\Users\SMJ\.conda\envs\yolov3-tf2-gpu\lib\site-packages\ipykernel_launcher.py, Second from C:\Users\SMJ\.conda\envs\yolov3-tf2-gpu\lib\site-packages\ipykernel_launcher.py.  Description from first occurrence: path to classes file

In [51]:
path = 'D:/STUDY/CV/yolov3-tf2/'
FLAGS.image = path + 'data/meme.jpg'

In [52]:
if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)
else:
    yolo = YoloV3(classes=FLAGS.num_classes)
    
yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weight loaded')

class_names = [c.strip() for c in open(FLAGS.classes).readline()]
logging.info('classes loaded')

I0131 10:02:39.604644  9596 3387376074.py:7] weight loaded
I0131 10:02:39.605623  9596 3387376074.py:10] classes loaded


In [53]:
img_raw = tf.image.decode_image(open(FLAGS.image, 'rb').read(), channels=3)

img = tf.expand_dims(img_raw, 0)
img = transform_images(img, FLAGS.size)

t1 = time.time()
boxes, scores, classes, nums = yolo(img)
t2 = time.time()
logging.info('time: {}'.format(t2 - t1))

logging.info('detections:')
for i in range(nums[0]):
    logging.info('{}, {}, {}'.format(class_names[int(classes[i])],
                                      np.array(scores[0][i]),
                                      np.array(boxes[0][i])))
    
img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

I0131 10:02:43.121371  9596 3700512197.py:9] time: 0.31200361251831055
I0131 10:02:43.122371  9596 3700512197.py:11] detections:


IndexError: list index out of range

In [43]:
from IPython.display import Image, display

display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))

TypeError: Expected Ptr<cv::UMat> for argument 'img'